# Modelo de deterioro de cuentas por cobrar de acuerdo con la NIIF 9

## Cargue de los datos

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agrega la ruta del directorio 'src' al path

sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np

#Librerias Machine Learning y modelado
from pycaret.classification import *
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

#Librerias para visualización de datos
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from procesamiento_datos import *

Función de procesamiento de datos cargadas correctamente.


In [2]:
# cargar los dataset 

ruta_base = os.path.abspath(os.path.join(os.getcwd(), '..'))

df_cierres = pd.read_csv(ruta_base+"/data/raw/Historico_Cierres_Tipo.csv")
df_variables_macro = pd.read_csv(ruta_base+"/data/raw/Variables_Macro.csv")
df_score = pd.read_csv(ruta_base+"/data/processed/datos_modelado.csv")

In [3]:
df_cierres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fecha_cierre          13079 non-null  object 
 1   NIT                   13079 non-null  object 
 2   Fecha_vencimiento     13079 non-null  object 
 3   Dias_mora             13079 non-null  int64  
 4   Calificación          13079 non-null  object 
 5   Saldo                 13079 non-null  float64
 6   Porcentaje_deterioro  13079 non-null  float64
 7   Deterioro             13079 non-null  float64
 8   FID                   13079 non-null  object 
 9   nombre_linea          13079 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 1021.9+ KB


In [4]:
convertir_fecha(df_cierres,['Fecha_vencimiento','fecha_cierre'])
df_cierres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   fecha_cierre          13079 non-null  datetime64[ns]
 1   NIT                   13079 non-null  object        
 2   Fecha_vencimiento     13079 non-null  datetime64[ns]
 3   Dias_mora             13079 non-null  int64         
 4   Calificación          13079 non-null  object        
 5   Saldo                 13079 non-null  float64       
 6   Porcentaje_deterioro  13079 non-null  float64       
 7   Deterioro             13079 non-null  float64       
 8   FID                   13079 non-null  object        
 9   nombre_linea          13079 non-null  object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(4)
memory usage: 1021.9+ KB


In [5]:
df_variables_macro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fecha_cierre      906 non-null    object 
 1   smmlv             498 non-null    float64
 2   ipc               318 non-null    float64
 3   pib               246 non-null    float64
 4   tasa              328 non-null    float64
 5   var_pct_ipc_1m    906 non-null    float64
 6   var_pct_pib_1m    906 non-null    float64
 7   var_pct_tasa_1m   906 non-null    float64
 8   var_pct_smmlv_1m  906 non-null    float64
dtypes: float64(8), object(1)
memory usage: 63.8+ KB


In [6]:
convertir_fecha(df_variables_macro,['fecha_cierre'])
df_variables_macro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_cierre      906 non-null    datetime64[ns]
 1   smmlv             498 non-null    float64       
 2   ipc               318 non-null    float64       
 3   pib               246 non-null    float64       
 4   tasa              328 non-null    float64       
 5   var_pct_ipc_1m    906 non-null    float64       
 6   var_pct_pib_1m    906 non-null    float64       
 7   var_pct_tasa_1m   906 non-null    float64       
 8   var_pct_smmlv_1m  906 non-null    float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 63.8 KB


In [7]:

df_cierres_agg = df_cierres.merge(df_variables_macro, on= 'fecha_cierre', how='left')
df_cierres_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   fecha_cierre          13079 non-null  datetime64[ns]
 1   NIT                   13079 non-null  object        
 2   Fecha_vencimiento     13079 non-null  datetime64[ns]
 3   Dias_mora             13079 non-null  int64         
 4   Calificación          13079 non-null  object        
 5   Saldo                 13079 non-null  float64       
 6   Porcentaje_deterioro  13079 non-null  float64       
 7   Deterioro             13079 non-null  float64       
 8   FID                   13079 non-null  object        
 9   nombre_linea          13079 non-null  object        
 10  smmlv                 13079 non-null  float64       
 11  ipc                   13079 non-null  float64       
 12  pib                   13079 non-null  float64       
 13  tasa            

In [8]:
df_clientes = pd.read_csv(ruta_base+'/data/raw/Informacion_Clientes.csv')
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   NIT                        2697 non-null   object 
 1   PERIODO                    2697 non-null   object 
 2   FECHA_CORTE                2697 non-null   object 
 3   ACTIVOS_CORRIENTES         2697 non-null   float64
 4   GANANCIAS                  2697 non-null   float64
 5   PATRIMONIO                 2697 non-null   float64
 6   TOTAL_ACTIVOS              2697 non-null   float64
 7   ACTIVOS_NO_CORRIENTES      2697 non-null   float64
 8   PASIVOS_NO_CORRIENTES      2697 non-null   float64
 9   TOTAL_PASIVOS              2697 non-null   float64
 10  VAR_ACTIVOS_CORRIENTES     2697 non-null   float64
 11  VAR_GANANCIAS              2696 non-null   float64
 12  VAR_PATRIMONIO             2697 non-null   float64
 13  VAR_ACTIVOS                2697 non-null   float

In [9]:
df_cierres_cliente = df_cierres_agg.copy()

In [10]:
df_cierres_cliente = df_cierres_cliente.merge(df_clientes,on='NIT',how='left')
df_cierres_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   fecha_cierre               13079 non-null  datetime64[ns]
 1   NIT                        13079 non-null  object        
 2   Fecha_vencimiento          13079 non-null  datetime64[ns]
 3   Dias_mora                  13079 non-null  int64         
 4   Calificación               13079 non-null  object        
 5   Saldo                      13079 non-null  float64       
 6   Porcentaje_deterioro       13079 non-null  float64       
 7   Deterioro                  13079 non-null  float64       
 8   FID                        13079 non-null  object        
 9   nombre_linea               13079 non-null  object        
 10  smmlv                      13079 non-null  float64       
 11  ipc                        13079 non-null  float64       
 12  pib 

In [11]:
df_cierres_cliente = df_cierres_cliente.merge(df_score[['NIT','score_credito']],on='NIT', how='left')
df_cierres_cliente.head()

,fecha_cierre,NIT,Fecha_vencimiento,Dias_mora,Calificación,Saldo,Porcentaje_deterioro,Deterioro,FID,nombre_linea,...,VAR_PASIVOS_NO_CORRIENTES,VAR_TOTAL_PASIVOS,PASIVOS_CORRIENTES,VAR_PASIVOS_CORRIENTES,UTILIDAD_NETA,LIQUIDEZ_CORRIENTE,CAPITAL_TRABAJO,ENDEUDAMIENTO,ROA,score_credito
0,2023-01-31,bda703a41ee55dcb911f17164009bc87636552bd5a641b...,2022-09-15,0,A,2380000.0,0.0022,5236.0,b527ab75d12cba5a48db8e39b6dc776fafe8d0a64454c1...,Inmobiliario,...,528.398825,-9.654030,6997878.0,-87.263588,2728080.0,7.13,42878314.0,1.00,4.81,100.0
1,2023-01-31,a6f8bc5c429a36aba24d818a811da81ef5f66107c131f4...,2023-02-28,0,A,0.0,0.0022,0.0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Otros,...,-19.458861,-31.443608,1228610.0,-77.468232,-423509.0,1.51,624676.0,1.17,-2.73,84.0
2,2023-01-31,f810167a3f88559f1cf90ac412c7d70c0de7f54742bcad...,2023-02-28,0,A,0.0,0.0022,0.0,eaab7f29f0ebeeb28a5fcae7dba2f7b07bf2fa942bab4b...,Administración,...,248.973746,67.151176,7223354.0,-39.709844,-2653178.0,2.93,13976680.0,0.76,-6.34,78.9
3,2023-01-31,c8ad84d88132c613b3343688e13ada573e8976d18b58ec...,2023-02-28,0,A,0.0,0.0022,0.0,aa6df8861c8ad85418960d463c1bfff2ec9d257d58e5ab...,Fondos de inversión colectiva,...,inf,366.503959,2880835.0,125.100582,-422714.0,3.22,6396260.0,0.64,-4.50,91.4
4,2023-01-31,c5cd27ae8413be0c194958c8ad37774a81b07873ab2642...,2023-02-28,0,A,0.0,0.0022,0.0,f4b94821159e4974022084b6d3735e1ff440cab7d97ce5...,Inmobiliario,...,121.365063,121.365063,0.0,NaN,-26634.0,inf,3888635.0,0.67,-0.20,99.7


In [12]:
df_cierres_cliente['default'] = df_cierres_cliente.apply(lambda row : 1 if row['Dias_mora'] > 90 or row['score_credito'] < 30 else 0, axis=1 )
print(df_cierres_cliente['default'].value_counts())

default
0    11133
1     1946
Name: count, dtype: int64


## Modelamiento

In [13]:
import pandas as pd
import numpy as np

#Librerias Machine Learning y modelado
from pycaret.classification import *
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

#Librerias para visualización de datos
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [14]:
df_modelo = df_cierres_cliente.copy()

In [15]:
X = df_modelo.drop(columns = ['default','NIT','fecha_cierre', 'Fecha_vencimiento','Dias_mora','Calificación','FID','FECHA_CORTE','PERIODO','score_credito','Porcentaje_deterioro'])
y = df_modelo['default'] 

In [16]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, stratify=y, random_state=123)

train_data = pd.concat([train_X, train_y], axis=1)
test_data = pd.concat([test_X, test_y], axis=1)

In [17]:
train_data_replaced = train_data.replace([np.inf, -np.inf], np.nan)
test_data_replaced = test_data.replace([np.inf, -np.inf], np.nan)

In [18]:
clf = setup(
    data=train_data_replaced,
    target='default',
    session_id=123,
    normalize=True,
    fix_imbalance=True,
    fix_imbalance_method='SMOTE',
    fold=5,
    categorical_features=['nombre_linea'],
    verbose=True,
    remove_multicollinearity=True
    )

,Description,Value
0,Session id,123
1,Target,default
2,Target type,Binary
3,Original data shape,"(10463, 33)"
4,Transformed data shape,"(15607, 30)"
5,Transformed train set shape,"(12468, 30)"
6,Transformed test set shape,"(3139, 30)"
7,Numeric features,31
8,Categorical features,1
9,Rows with missing values,18.4%


In [19]:
best_model = compare_models(include=['xgboost','dt','lr','rf'],fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9964,0.9970,0.9789,0.9972,0.9880,0.9859,0.9859,2.2960
rf,Random Forest Classifier,0.9960,0.9971,0.9752,0.9981,0.9865,0.9842,0.9843,1.3940
dt,Decision Tree Classifier,0.9877,0.9818,0.9734,0.9457,0.9593,0.9520,0.9522,0.3580
lr,Logistic Regression,0.9427,0.9349,0.7303,0.8636,0.7911,0.7582,0.7619,0.4060


In [20]:
tuned_best_model = tune_model(best_model, fold = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9959,0.9957,0.9908,0.9818,0.9863,0.9839,0.9839
1,0.9952,0.9969,0.9908,0.9774,0.9841,0.9812,0.9813
2,0.9973,0.9973,0.9908,0.9908,0.9908,0.9892,0.9892
3,0.9980,0.9963,0.9954,0.9909,0.9931,0.9919,0.9919
4,0.9911,0.9948,0.9817,0.9596,0.9705,0.9653,0.9654
Mean,0.9955,0.9962,0.9899,0.9801,0.9850,0.9823,0.9823
Std,0.0024,0.0009,0.0045,0.0115,0.0079,0.0093,0.0093


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [21]:
predict_model (tuned_best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.9968,0.9958,0.9786,1.0000,0.9892,0.9873,0.9874


,Saldo,Deterioro,nombre_linea,smmlv,ipc,pib,tasa,var_pct_ipc_1m,var_pct_pib_1m,var_pct_tasa_1m,...,PASIVOS_CORRIENTES,VAR_PASIVOS_CORRIENTES,UTILIDAD_NETA,LIQUIDEZ_CORRIENTE,CAPITAL_TRABAJO,ENDEUDAMIENTO,ROA,default,prediction_label,prediction_score
10764,0.000000e+00,0.000000,Inmobiliario,1423500.0,5.49,452132.03125,19.650000,0.733945,1.464090,0.357508,...,131613.0,-6.779757,-11346.0,2.520000,200164.0,0.11,-0.320000,0,0,0.9941
9431,3.691767e+06,8121.886719,Inmobiliario,1300000.0,5.91,445607.90625,20.299999,-4.368932,3.683884,-3.010033,...,64492732.0,-6.675771,-442450.0,2.130000,72868368.0,0.77,-0.240000,0,0,0.9998
310,0.000000e+00,0.000000,Administración,1160000.0,11.84,394716.62500,30.840000,2.510823,3.754114,7.419018,...,94832248.0,0.571075,-8762880.0,1.570000,53638896.0,0.37,-1.240000,0,0,0.9970
12781,1.270474e+06,2795.042236,Inmobiliario,1423500.0,5.12,453675.56250,19.059999,-2.476191,0.341389,-1.192328,...,131613.0,-6.779757,-11346.0,2.520000,200164.0,0.11,-0.320000,0,0,0.9998
5082,4.141200e+06,9110.639648,Administración,1160000.0,9.46,400311.96875,26.080000,-3.861789,1.062036,-0.381971,...,1581.0,-49.569378,-51127.0,1528.900024,2415614.0,0.00,-1.080000,0,0,0.9995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2.302507e+05,506.551575,Inmobiliario,1160000.0,12.16,394716.62500,29.950001,2.702703,3.754114,-2.885863,...,77440.0,-75.443626,312627.0,9.140000,630595.0,0.04,5.440000,0,0,0.9997
575,1.380400e+06,3036.879883,Inversión,1160000.0,12.16,394716.62500,29.950001,2.702703,3.754114,-2.885863,...,137875936.0,-5.882557,10484279.0,1.030000,3527261.0,0.59,3.960000,0,0,1.0000
3571,1.380400e+06,3589.040039,Inmobiliario,1160000.0,10.85,396105.18750,26.360001,-3.726708,0.680421,-4.630970,...,18934824.0,5.334960,163833.0,1.940000,17818112.0,0.50,0.430000,0,0,0.9992
7164,2.320500e+06,5105.100098,Inmobiliario,1300000.0,7.17,421222.62500,22.250000,-2.182810,2.777054,-3.762976,...,102755616.0,12.200352,214504.0,3.680000,275023200.0,0.82,0.050000,0,0,0.9999


In [22]:
evaluate_model(tuned_best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
correlacion = df_cierres_cliente.select_dtypes(include='number')

plt.figure(figsize=(12,8))
sns.heatmap(correlacion.corr(), annot=True,fmt='.2f',cmap='coolwarm',center=0,linewidths=0.5, annot_kws={"size":5})
plt.tight_layout()
plt.show()

# --- Stage ---

In [ ]:
# STAGE

df_cierres_cliente['Etapa_actual'] = df_cierres_cliente['Dias_mora'].apply(lambda x: 1 if x <= 90 else (2 if x <= 360 else 3 ))
df_cierres_cliente['Etapa_NIIF'] = df_cierres_cliente['Dias_mora'].apply(lambda x: 1 if x == 0 else (2 if x <= 90 else 3 ))

In [ ]:
etapa_actual = df_cierres_cliente.groupby(['Etapa_actual']).agg(cantidad = ('Etapa_actual','count'), saldo = ('Saldo','sum'), deterioro =('Deterioro','sum')).reset_index()
print (etapa_actual)

In [ ]:
etapa_NIIF = df_cierres_cliente.groupby(['Etapa_NIIF']).agg(cantidad = ('Etapa_NIIF','count'), saldo = ('Saldo','sum'),deterioro =('Deterioro','sum')).reset_index()
print (etapa_NIIF)

In [ ]:
diferencias = etapa_actual.merge(etapa_NIIF, left_on='Etapa_actual', right_on='Etapa_NIIF', how='left')
diferencias['Diferencia_cant'] = diferencias['cantidad_x']-diferencias['cantidad_y']
diferencias['Diferencia_saldo'] = diferencias['saldo_x']-diferencias['saldo_y']
diferencias['Diferencia_deterioro'] = diferencias['deterioro_x']-diferencias['deterioro_y']
diferencias['porc_deteriorado_x'] = diferencias['deterioro_x']/diferencias['saldo_x']
diferencias['porc_deteriorado_y'] = diferencias['deterioro_y']/diferencias['saldo_y']
print(diferencias)